In [130]:
import sys
import os

# os.getcwd() => '/home/genie/Documents/Projekti/cash-flow/jupyter'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Moves up one level to 'cash-flow'
# project_root = '/home/genie/Documents/Projekti/cash-flow'

# Add the module Converters directory to sys.path
sys.path.append(os.path.join(project_root, 'cash_flow', 'util'))

In [131]:
import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine

from Converters import date_format

engine = create_engine("sqlite:///../data/database.db", echo=False)

In [132]:
# Setup filters
date_from = "2025-01-01"
date_through = "2025-03-31"
report_period = 'W-SUN'

In [133]:
budget_entries_df = pd.read_sql_query("SELECT * FROM F01_BudgetEntries WHERE date >= '" + date_from + "' AND date <= '" + date_through + "'", engine)
definition_df = pd.read_sql_query("SELECT * FROM E01_CashFlowDefinition WHERE definition_type = 1", engine)

In [134]:
budget_entries_df.head()

,id,definition_id,date,amount_LC
0,4,3,2025-01-10 00:00:00.000000,5647.46
1,3,31,2025-03-05 00:00:00.000000,71.55
2,5,31,2025-03-18 00:00:00.000000,1002.98
3,2,31,2025-03-19 00:00:00.000000,8004.25


In [135]:
definition_df.head()

,id,key,definition_type,name
0,2,101,1,Ieņēmumi no preču un pakalpojumu pārdošanas
1,3,102,1,Maksājumi piegādātājiem
2,5,104,1,Pārējie pamatdarbības ieņēmumi un izdevumi
3,7,201,1,Izdevumi procentu maksājumiem
4,8,202,1,Izdevumi nodokļu maksājumiem


In [136]:
budget_entries_df["date"] = pd.to_datetime(budget_entries_df['date'])
budget_entries_df["period"]=budget_entries_df["date"].dt.to_period(report_period).apply(lambda r: r.to_timestamp(how='end').normalize() if pd.notna(r) else pd.NaT)

In [137]:
budget_entries_df.head()

,id,definition_id,date,amount_LC,period
0,4,3,2025-01-10,5647.46,2025-01-12
1,3,31,2025-03-05,71.55,2025-03-09
2,5,31,2025-03-18,1002.98,2025-03-23
3,2,31,2025-03-19,8004.25,2025-03-23


In [138]:
pivot_df = budget_entries_df.pivot_table(
    index='definition_id',
    columns='period',
    values='amount_LC',
    aggfunc="sum",
    fill_value=0,
    dropna=False
)


In [139]:
pivot_df

period,2025-01-12,2025-03-09,2025-03-23
definition_id,,,
3,5647.46,0.00,0.00
31,0.00,71.55,9007.23


In [140]:
all_periods = pd.date_range(start=date_from, end=date_through, freq=report_period)
pivot_df = pivot_df.reindex(columns=all_periods, fill_value=0)

In [142]:
# Ensure columns are in datetime format before formatting
pivot_df.columns = pd.to_datetime(pivot_df.columns, errors='coerce')
# Format column headers to show only the date part
pivot_df.columns = [col.strftime(date_format()) if not pd.isnull(col) else col for col in pivot_df.columns]

In [143]:
pivot_df

,"05. Jan, 2025","12. Jan, 2025","19. Jan, 2025","26. Jan, 2025","02. Feb, 2025","09. Feb, 2025","16. Feb, 2025","23. Feb, 2025","02. Mar, 2025","09. Mar, 2025","16. Mar, 2025","23. Mar, 2025","30. Mar, 2025"
definition_id,,,,,,,,,,,,,
3,0,5647.46,0,0,0,0,0,0,0,0.00,0,0.00,0
31,0,0.00,0,0,0,0,0,0,0,71.55,0,9007.23,0


In [149]:
budget_df = pd.merge(definition_df, pivot_df, 
                     left_on="id", right_on="definition_id", how="left").fillna(0)
budget_df.sort_values("key", inplace=True)
budget_df.drop(columns=["definition_type", "key"], inplace=True)
budget_df.set_index("id", inplace=True)

In [150]:
budget_df.head()

,name,"05. Jan, 2025","12. Jan, 2025","19. Jan, 2025","26. Jan, 2025","02. Feb, 2025","09. Feb, 2025","16. Feb, 2025","23. Feb, 2025","02. Mar, 2025","09. Mar, 2025","16. Mar, 2025","23. Mar, 2025","30. Mar, 2025"
id,,,,,,,,,,,,,,
2,Ieņēmumi no preču un pakalpojumu pārdošanas,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
3,Maksājumi piegādātājiem,0.0,5647.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
31,Maksājumi darbiniekiem,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.55,0.0,9007.23,0.0
5,Pārējie pamatdarbības ieņēmumi un izdevumi,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
7,Izdevumi procentu maksājumiem,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
